# Tarefa 2b: Resumir textos abstrativos

Neste caderno, você gerencia os desafios decorrentes do resumo de documentos grandes. O texto de entrada pode exceder o tamanho do contexto do modelo, gerar saídas alucinadas ou acionar erros de falta de memória.

Para mitigar esses problemas, esse caderno demonstra uma arquitetura que usa fragmentação e encadeamento de prompts com o framework [LangChain] (https://python.langchain.com/docs/get_started/introduction.html), um toolkit que permite que aplicações usem modelos de linguagem.

Você verá uma abordagem de cenários em que os documentos do usuário ultrapassam os limites de tokens. A fragmentação divide os documentos em segmentos menores do que os limites de comprimento do contexto antes de alimentá-los sequencialmente nos modelos. Isso encadeia prompts em blocos, mantendo o contexto anterior. Você aplica essa abordagem para resumir transcrições de chamadas, transcrições de reuniões, livros, artigos, publicações de blogs e outros conteúdos relevantes.

## Tarefa 2b.1: Configurar o ambiente

Nesta tarefa, você vai configurar o ambiente.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## Tarefa 2b.2: Resumir texto longo 

### Configurar o LangChain com o Boto3

Nessa tarefa, você precisa especificar o LLM para a classe do Bedrock do LangChain e pode transmitir argumentos para inferência.

In [ ]:
# model configuration
from langchain_aws import BedrockLLM
modelId = "amazon.titan-text-premier-v1:0"
llm = BedrockLLM(
    model_id=modelId,
    model_kwargs={
        "maxTokenCount": 2048,
        "temperature": 0,
        "topP": 1
    },
    client=bedrock_client
)

## Tarefa 2b.3: Carregar um arquivo de texto com muitos tokens

Nesta tarefa, você pode encontrar um arquivo de texto da [carta do CEO da Amazon aos acionistas em 2022] (https://www.aboutamazon.com/news/company-news/amazon-ceo-andy-jassy-2022-letter-to-shareholders) no diretório de cartas. A célula a seguir carrega o arquivo de texto e conta o número de tokens. Você verá um aviso indicando que o número de tokens no arquivo de texto excede o número máximo de tokens para esse modelo.

In [ ]:
#get tokens
shareholder_letter = "../letters/2022-letter.txt"

with open(shareholder_letter, "r") as file:
    letter = file.read()
    
llm.get_num_tokens(letter)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Observação: ** você pode ignorar os avisos com segurança e prosseguir para a próxima célula.

## Tarefa 2b.4: Dividir o texto longo em blocos

Nesta tarefa, você divide o texto em blocos menores porque ele é muito longo para caber no prompt. `RecursiveCharacterTextSplitter` no LangChain permite dividir recursivamente o texto longo em blocos até que o tamanho de cada bloco seja menor que `chunk_size`. Um texto é separado com `separators=["\n\n", "\n"]` em blocos, o que evita a divisão de cada parágrafo em vários blocos.

Ao usar 6 mil caracteres por bloco, você pode resumir cada parte separadamente. O número de tokens, ou pedaços de palavras, em um bloco dependerá do texto.

In [ ]:
#chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [ ]:
num_docs = len(docs)

num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

print(
    f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
)

## Tarefa 2b.5: Resumir e combinar blocos

Nesta tarefa, supondo que o número de tokens seja consistente nos outros documentos, você deve ter tudo pronto para começar. Você pode usar o `load_summarize_chain` do LangChain para resumir o texto. `load_summarize_chain` oferece três formas de resumo: `stuff`, `map_reduce` e `refine`.

- `stuff`: coloca todos os blocos em um só prompt. Sendo assim, isso atingiria o limite máximo de tokens.
- `map_reduce`: resume cada bloco, combina os resumos e sintetiza o resumo combinado. Quando o resumo combinado é muito grande, isso gera um erro.
- `refine`: resume o primeiro bloco e depois o segundo bloco usando o primeiro resumo. O mesmo processo se aplica até que todos os blocos sejam resumidos.

Tanto map_reduce quanto refine invocam o LLM várias vezes e levam tempo para obter o resumo final. Você pode tentar map_reduce aqui.

In [ ]:
# Set verbose=True if you want to see the prompts being used
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=False)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Observação:** dependendo do número de documentos, da cota de taxa de solicitações do Bedrock e das configurações de novas tentativas definidas, o processamento da cadeia abaixo pode levar algum tempo.

In [ ]:
#invoke chain
output = ""
try:
    
    output = summary_chain.invoke(docs)

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

In [ ]:
# print output
print(output['output_text'])

Você testou o uso da fragmentação e do encadeamento de prompts com o framework LangChain para resumir documentos grandes e, ao mesmo tempo, mitigar problemas decorrentes de textos longos de entrada.

### Experimente você mesmo
- Altere os prompts para seu caso de uso específico e avalie o resultado de diferentes modelos.
- Teste o comprimento do token para entender a latência e a responsividade do serviço.
- Aplique diferentes princípios de engenharia de prompts para gerar resultados melhores.

### Limpeza

Você concluiu este caderno. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de caderno e continue com a **Tarefa 3**.